In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 01:32:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 01:32:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/10 01:32:37 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/10 01:32:37 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/10 01:32:37 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


### 1. Connect Transactions and Merchants AS `sdf`

In [4]:
print("================ Transactions ================")
FILE_LIST = ['transactions_20210228_20210827_snapshot', 'transactions_20210828_20220227_snapshot', 'transactions_20220228_20220828_snapshot']
transactions_sdf = read_file(spark, FILE_LIST[0])

for path in FILE_LIST[1:]:
    temp_sdf = read_file(spark, path)
    transactions_sdf = transactions_sdf.union(temp_sdf)
temp = transactions_sdf.dropna(how='any').count()
print( f'There are {transactions_sdf.count()} data, and {temp} of them have no null value.')

================ Transactions ================
|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 18478                                
 merchant_abn   | 62191208634                          
 dollar_value   | 63.255848959735246                   
 order_id       | 949a63c8-29f7-4ab0-ada4-99ac50a88952 
 order_datetime | 2021-08-20                           
only showing top 1 row

|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 14935                                
 merchant_abn   | 79417999332                          
 dollar_value   | 136.06570809815838                   
 order_id       | 23acbb7b-cf98-4580-9775-86b8e0a2bd88 
 order_datetime | 2021-11-26                           
only showing top 1 row

|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 11139                                
 merchant_abn   | 96152467973                          
 dollar_value   | 16.213590228273233                   
 order_id       | 785b0080-9e4b-471a-9b1f-6a8a9dc535c2 
 order_datetime | 2022-08-20                           
only showing top 1 row



There are 14195505 data, and 14195505 of them have no null value.


In [5]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'merchants_data.parquet', '../data/curated/', truncate=20)
temp = merchants_sdf.dropna(how='any').count()
print( f'There are {merchants_sdf.count()} data, and {temp} of them have no null value.')

================ Merchant ================
|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------
 merchant_name | Felis Limited        
 tags          | furniture, home f... 
 merchant_abn  | 10023283211          
 take_rate     | 0.18                 
 type          | e                    
only showing top 1 row

There are 4026 data, and 4026 of them have no null value.


- Known that Transactions and Merchants table have no null value

In [6]:
sdf = transactions_sdf.join(merchants_sdf, how='left', on='merchant_abn')
temp = sdf.dropna(how='any', subset=['merchant_name']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 14195505 data, and 13614675 of them have no null value.


In [7]:
sdf_agg = sdf.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in sdf.columns])
sdf_agg

merchant_abn,user_id,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type
0,0,0,0,0,580830,580830,580830,580830


In [8]:
580830/14195505

0.040916473207539994

- There are 4026 merchants, and for all three batches data have 14195505 records. 

    After combination, there are **3494038** rest. **14195505 - 13614675 = 580830** records has been deleted. 
    
    `580830/14195505 = 0.040916473207539994`

In [9]:
sdf.show(5)

+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
|merchant_abn|user_id|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
| 62191208634|  18478|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|Cursus Non Egesta...|furniture, home f...|     2.17|   c|
| 15549624934|      2| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|  Commodo Associates|opticians, optica...|     2.76|   c|
| 64403598239|  18479|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|Lobortis Ultrices...|music shops - mus...|     6.31|   a|
| 60956456424|      3| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|Ultricies Digniss...|gift, card, novel...|     4.69|   b|
| 94493496784|  18479| 72.96316578355305|f6c78c1a-4600-

In [10]:
null_df = sdf.filter(sdf.merchant_name.isNull())
# 580830
print(null_df.count())
# 396
new_merchant = null_df.drop_duplicates(['merchant_abn'])
new_merchant.count()

580830


396

### 2. `sdf` join with consumer

In [11]:
print("================ Customer ================")
consumer = read_file(spark, 'new_consumer_data.parquet', '../data/curated/')
temp = consumer.dropna(how='any').count()
print( f'There are {consumer.count()} data, and {temp} of them have no null value.')

================ Customer ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------------------
 consumer_id | 680810                     
 name        | Courtney Mendez            
 address     | 802 Knapp Harbors Apt. 769 
 state       | QLD                        
 postcode    | 9013                       
 gender      | Female                     
 user_id     | 71144                      
only showing top 1 row

There are 499999 data, and 499999 of them have no null value.


- consumer join with poa population data

In [12]:
print("================  ================")
poa_sdf = read_file(spark, 'external_pos_population.parquet', '../data/curated/')

================  ================
|> Loading File...
|> Loading Finished!


-RECORD 0--------------------------------------------------------------------------------------
 postcode   | 6935                                                                             
 POA_CODE21 | 6055                                                                             
 geometry   | [01 03 00 00 00 01 00 00 00 F2 05 00 00 68 A7 F7 0B 98 FD 5C 40 C0 AB 6F 9D 5... 
 2020(no.)  | 33667.56962451867                                                                
 2021(no.)  | 35471.469825504624                                                               
 latitude   | 115.98678832411451                                                               
 longitude  | -31.863786871018736                                                              
only showing top 1 row



In [13]:
poa_sdf = poa_sdf.drop('geometry')
poa_sdf = poa_sdf.withColumnRenamed('2020(no.)', '2020_popu').withColumnRenamed('2021(no.)', '2021_popu')

In [14]:
poa_sdf.show(5)

+--------+----------+------------------+------------------+------------------+-------------------+
|postcode|POA_CODE21|         2020_popu|         2021_popu|          latitude|          longitude|
+--------+----------+------------------+------------------+------------------+-------------------+
|    6935|      6055| 33667.56962451867|35471.469825504624|115.98678832411451|-31.863786871018736|
|    1109|      2000| 31245.01595130939|29155.593195479523| 151.2082196508253|  -33.8683837718929|
|    6849|      6014| 23594.15360007898|23626.473167324344|115.80478169490398| -31.93687853219304|
|    1114|      2000| 31245.01595130939|29155.593195479523| 151.2082196508253|  -33.8683837718929|
|    1825|      2144|42033.877748936546|40303.381920963206|151.02620831540813| -33.85468695739709|
+--------+----------+------------------+------------------+------------------+-------------------+
only showing top 5 rows



In [15]:
temp = poa_sdf.dropna(how='any').count()
print( f'There are {poa_sdf.count()} data, and {temp} have no null value.')

There are 3158 data, and 3158 have no null value.


In [16]:
consumer = consumer.join(poa_sdf, on='postcode', how='left')

In [17]:
consumer.show(5)

+--------+-----------+---------------+--------------------+-----+------+-------+----------+------------------+------------------+------------------+-------------------+
|postcode|consumer_id|           name|             address|state|gender|user_id|POA_CODE21|         2020_popu|         2021_popu|          latitude|          longitude|
+--------+-----------+---------------+--------------------+-----+------+-------+----------+------------------+------------------+------------------+-------------------+
|    8205|     699376|Jeffrey Harding|665 Doyle Oval Ap...|  VIC|  Male|  71147|      3000| 51550.57421324455| 44766.40475584961|144.96238093177988| -37.81314234745685|
|    5201|    1440589|     Adam Bowen|77454 Lopez Plaza...|   SA|  Male|  71146|      5201|2560.1036151846665|2615.9481608550705| 138.7323332150744| -35.20611204854585|
|    9013|     680810|Courtney Mendez|802 Knapp Harbors...|  QLD|Female|  71144|      4000|21606.625243521717|  21142.3135518536|153.02369142357665|-27.465

In [18]:
consumer.dropna(how='any').count()

498587

In [19]:
# 有9个postcode没有找到对应的数据
# 就是做预处理找过的 --- 保留到fraud做处理
consumer[consumer.POA_CODE21.isNull()].drop_duplicates(['postcode'])

postcode,consumer_id,name,address,state,gender,user_id,POA_CODE21,2020_popu,2021_popu,latitude,longitude
1441,341498,Courtney Alexander,82265 Hardy Hill,NSW,Female,77783,null,null,null,null,null
1476,588776,Elizabeth Sanchez,343 Middleton Shoal,NSW,Female,73404,null,null,null,null,null
1819,1131147,Marc Wilson,162 Shields Inlet...,NSW,Male,72313,null,null,null,null,null
2013,861495,Erin Rodriguez,707 Xavier Brook,NSW,Female,75224,null,null,null,null,null
3399,724573,Andrea Stone,627 Christine Cor...,VIC,Female,72496,null,null,null,null,null
3989,1093754,Tracey Williams,49197 Nunez Burg,VIC,Female,71746,null,null,null,null,null
6435,1217055,Douglas Jones,71897 Kristina Hi...,WA,Undisclosed,71334,null,null,null,null,null
8785,806615,Destiny Morgan,68138 Welch Divid...,VIC,Female,72252,null,null,null,null,null
9999,545212,Daniel Walker,2217 Fleming Rest...,VIC,Male,72597,null,null,null,null,null


In [20]:
consumer = consumer.na.fill(subset='2021_popu', value = 0)
consumer = consumer.na.fill(subset='2020_popu', value = 0)

In [21]:
consumer = consumer.na.fill(subset='POA_CODE21', value = 0)

In [22]:
mean = consumer.agg(*(F.mean(c).alias(c) for c in consumer.columns))
meaninfo = mean.first().asDict()
meaninfo

{'postcode': 4037.0854181708364,
 'consumer_id': 750895.2123184246,
 'name': None,
 'address': None,
 'state': None,
 'gender': None,
 'user_id': 250000.0,
 'POA_CODE21': 4003.05151010302,
 '2020_popu': 12533.471767338458,
 '2021_popu': 12385.711385835375,
 'latitude': 142.10512461626408,
 'longitude': -32.99312950584857}

In [23]:
consumer = consumer.na.fill(subset='latitude', value = meaninfo['latitude'])
consumer = consumer.na.fill(subset='longitude', value = meaninfo['longitude'])

In [24]:
consumer[consumer.longitude.isNull()].drop_duplicates(['postcode'])

postcode,consumer_id,name,address,state,gender,user_id,POA_CODE21,2020_popu,2021_popu,latitude,longitude


- join


In [25]:
sdf

merchant_abn,user_id,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type
62191208634,18478,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20,Cursus Non Egesta...,"furniture, home f...",2.17,c
15549624934,2,130.3505283105634,6a84c3cf-612a-457...,2021-08-20,Commodo Associates,"opticians, optica...",2.76,c
64403598239,18479,120.15860593212783,b10dcc33-e53f-425...,2021-08-20,Lobortis Ultrices...,music shops - mus...,6.31,a
60956456424,3,136.6785200286976,0f09c5a5-784e-447...,2021-08-20,Ultricies Digniss...,"gift, card, novel...",4.69,b
94493496784,18479,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20,Dictum Phasellus ...,"gift, card, novel...",5.65,a
76819856970,3,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20,Egestas Blandit Ltd,tent and awning s...,3.19,b
67609108741,18479,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20,Metus Sit Amet In...,"cable, satellite,...",0.38,e
34096466752,3,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20,Nullam Enim Ltd,"computers, comput...",3.22,b
70501974849,18482,68.75486276223054,8505fb33-b69a-412...,2021-08-20,Facilisis Lorem T...,"computers, comput...",3.3,b
49891706470,4,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20,Non Vestibulum In...,tent and awning s...,5.8,a


In [26]:
sdf = sdf.join(consumer, how='left', on='user_id')
temp = sdf.dropna(how='any', subset=['consumer_id']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 14195505 data, and 14195505 of them have no null value.


In [27]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+-------------+--------------------+-----+------+----------+-----------------+-----------------+------------------+-------------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|postcode|consumer_id|         name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|          latitude|          longitude|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+-------------+--------------------+-----+------+----------+-----------------+-----------------+------------------+-------------------+
|      7| 96442520325|29.863817960377617|48dd50dd-291d-43d...|    2021-04-07|Nullam Lobortis Q...|computers, comput...|     3.65| 

### 3. `sdf` join with the external dataset  --- ncd

In [28]:
print("================  ================")
ncd_sdf = read_file(spark, 'external_ncd.parquet', '../data/curated/')
temp = ncd_sdf.dropna(how='any').count()
print( f'There are {ncd_sdf.count()} data, and {temp} have no null value.')

================  ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------
 Date            | 2020-01-25 
 New cases / day | 4          
only showing top 1 row

There are 980 data, and 980 have no null value.


In [29]:
ncd_sdf.show(5)

+----------+---------------+
|      Date|New cases / day|
+----------+---------------+
|2020-01-25|              4|
|2020-01-26|              0|
|2020-01-27|              1|
|2020-01-28|              0|
|2020-01-29|              4|
+----------+---------------+
only showing top 5 rows



In [30]:
sdf = sdf.join(ncd_sdf, sdf.order_datetime == ncd_sdf.Date, 'left')
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+-------------+--------------------+-----+------+----------+-----------------+-----------------+------------------+-------------------+----------+---------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|postcode|consumer_id|         name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|          latitude|          longitude|      Date|New cases / day|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+-------------+--------------------+-----+------+----------+-----------------+-----------------+------------------+-------------------+----------+---------------+
|      7| 96442520325|29.863817960377617|48dd50dd

In [31]:
temp = sdf.dropna(how='any', subset=['Date']).count()
print( f'There are {sdf.count()} data, and {temp} have no null value.')

There are 14213202 data, and 13458582 have no null value.


In [32]:
sdf = sdf.drop('Date')

In [33]:
# temp = sdf.to_pandas_on_spark()
# temp[temp.Date.isna()]

### 4. join with external retail trade seasonal

In [34]:
print("================  ================")
rts_sdf = read_file(spark, 'external_retail_trade_seasonal.parquet', '../data/curated/')
temp = rts_sdf.dropna(how='any').count()
print( f'There are {rts_sdf.count()} data, and {temp} have no null value.')

================  ================
|> Loading File...
|> Loading Finished!
-RECORD 0----------------------------------------------
 month                                    | 2022-04-01 
 food                                     | 13182.7    
 household goods                          | 6023.3     
 clothing footwear and personal accessory | 2912.8     
 department store                         | 1782.5     
 other                                    | 5317.8     
 cafes restaurants and takeway foods      | 4704       
 total                                    | 33923.1    
only showing top 1 row

There are 19 data, and 19 have no null value.


In [35]:
rts_sdf.printSchema()

root
 |-- month: string (nullable = true)
 |-- food: string (nullable = true)
 |-- household goods: string (nullable = true)
 |-- clothing footwear and personal accessory: string (nullable = true)
 |-- department store: string (nullable = true)
 |-- other: string (nullable = true)
 |-- cafes restaurants and takeway foods: string (nullable = true)
 |-- total: string (nullable = true)



In [36]:
rts_sdf = rts_sdf.select(['month', 'total'])

In [37]:
sdf = sdf.join(rts_sdf, (F.months_between(sdf.order_datetime, rts_sdf.month) < 1) & (F.months_between(sdf.order_datetime, rts_sdf.month) >= 0), 'left')

In [38]:
temp = sdf.dropna(how='any', subset=['month']).count()
print( f'There are {sdf.count()} data, and {temp} have no null value.')

There are 14213202 data, and 11831330 have no null value.


In [39]:
# sdf.show(5)

In [40]:
sdf = sdf.drop('2020_popu')

In [41]:
sdf[sdf.POA_CODE21.isNull()].drop_duplicates(['postcode'])

22/10/10 01:54:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type,postcode,consumer_id,name,address,state,gender,POA_CODE21,2021_popu,latitude,longitude,New cases / day,month,total


In [42]:
# sdf.show(5)

In [43]:
# from pyspark.sql.functions import datediff, months_between

# # 1.日期差
# df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
# df.select(datediff(df.d1, df.d2).alias('diff')).show()

# # 2. 月份差
# df = spark.createDataFrame([('1999-02-28 10:30:00', '1996-10-30')], ['t', 'd'])
# df.select(months_between(df.t, df.d).alias('months')).show()

In [45]:
path = '../data/curated/data'
sdf.write.partitionBy('order_datetime').parquet(path, mode='overwrite')

In [ ]:
# sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type,postcode,consumer_id,name,address,state,gender,POA_CODE21,2021_popu,latitude,longitude,New cases / day,month,total
7,33064796871,373.0873675184212,fe188788-b89f-4dd...,2021-08-20,Curabitur Massa C...,computer programm...,3.75,b,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,706,2021-08-01,29261.9
7,68435002949,232.5364986739752,b4a89891-a113-45e...,2021-08-20,Aliquam Eu Inc.,artist supply and...,6.65,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,706,2021-08-01,29261.9
7,41944909975,30.910755230234322,302ae628-8eba-4a5...,2021-08-20,Et Nunc Consulting,"books, periodical...",0.16,e,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,706,2021-08-01,29261.9
7,89726005175,42.507331805214285,a8e087f3-c713-45c...,2021-05-24,Est Nunc Consulting,tent and awning s...,6.01,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,10,2021-05-01,31007.7
7,96442520325,29.863817960377617,48dd50dd-291d-43d...,2021-04-07,Nullam Lobortis Q...,"computers, comput...",3.65,b,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,14,2021-04-01,30960
7,29566626791,80.43460573195489,13d12435-4f1d-4b6...,2021-04-07,null,null,null,null,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,14,2021-04-01,30960
7,15269431008,75.4571735676935,74adc91d-44a1-436...,2021-04-07,Nisi Limited,"gift, card, novel...",6.48,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,14,2021-04-01,30960
7,62224020443,212.60670786220882,3dfa8cb9-b785-40d...,2021-12-21,Hendrerit A Corpo...,"watch, clock, and...",6.64,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,"4,522",2021-12-01,31967.9
7,99275601275,111.48028697136225,58af5b18-f28b-464...,2021-10-14,Proin Velit Sed F...,"furniture, home f...",1.87,c,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,"2,683",2021-10-01,31132.7
7,26749181046,154.59228188209104,8673c0d9-1515-487...,2021-10-14,Neque Tellus Inco...,"watch, clock, and...",3.34,b,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,151.80006629311015,-26.268858171309105,"2,683",2021-10-01,31132.7


In [ ]:
sdf_agg = sdf.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in sdf.columns])
sdf_agg

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type,postcode,consumer_id,name,address,state,gender,POA_CODE21,2021_popu,latitude,longitude,New cases / day,month,total
0,0,0,0,0,581559,581559,581559,581559,0,0,0,0,0,0,0,0,0,0,754620,2381872,2381872


In [ ]:
sdf[sdf.month.isNull()].drop_duplicates(['order_datetime'])

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type,postcode,consumer_id,name,address,state,gender,POA_CODE21,2021_popu,latitude,longitude,New cases / day,month,total
26,89726005175,28.60583008450375,86c7eaee-3318-4d8...,2022-08-01,Est Nunc Consulting,tent and awning s...,6.01,a,3469,1421465,Kristi Adkins,051 Snyder Hollow...,VIC,Female,3469,495.0634561440711,143.26387846511204,-37.17472419953752,"34,112",null,null
19,34459028045,629.6095943808523,682a5310-8ab2-42f...,2022-08-02,Tellus Eu Inc.,"stationery, offic...",6.05,a,7276,1226530,Victoria Gonzalez,68657 Johnson Gle...,TAS,Female,7276,108.99254564781535,146.9635580479313,-41.28259951036976,"35,505",null,null
26,38523562929,109.86688348566284,a1ab64e6-4c6b-40f...,2022-08-03,Sed Dictum Corp.,"cable, satellite,...",6.56,a,3469,1421465,Kristi Adkins,051 Snyder Hollow...,VIC,Female,3469,495.0634561440711,143.26387846511204,-37.17472419953752,"41,014",null,null
19,94493496784,66.43822913988639,e5f8afbb-7fda-4a7...,2022-08-04,Dictum Phasellus ...,"gift, card, novel...",5.65,a,7276,1226530,Victoria Gonzalez,68657 Johnson Gle...,TAS,Female,7276,108.99254564781535,146.9635580479313,-41.28259951036976,"37,206",null,null
19,80779820715,18.315831932281192,152041f3-5d0a-456...,2022-08-05,Euismod Enim LLC,"watch, clock, and...",4.71,b,7276,1226530,Victoria Gonzalez,68657 Johnson Gle...,TAS,Female,7276,108.99254564781535,146.9635580479313,-41.28259951036976,"32,607",null,null
19,31397959824,204.7083359839957,66227d13-a66b-4d1...,2022-08-06,Blandit Congue In...,"books, periodical...",6.89,a,7276,1226530,Victoria Gonzalez,68657 Johnson Gle...,TAS,Female,7276,108.99254564781535,146.9635580479313,-41.28259951036976,"28,699",null,null
26,85139489422,30.03349461165711,d202d0ab-994f-407...,2022-08-07,Ut Institute,"cable, satellite,...",4.25,b,3469,1421465,Kristi Adkins,051 Snyder Hollow...,VIC,Female,3469,495.0634561440711,143.26387846511204,-37.17472419953752,"22,847",null,null
155,29639699851,561.4443737909377,f1398b22-b0d2-4d6...,2022-08-08,Sodales Elit Erat...,lawn and garden s...,6.21,a,4895,539251,Joseph Johnston,58700 Suzanne Rapid,QLD,Male,4895,2702.5590815949613,144.92385742559097,-15.139850882523248,"17,564",null,null
19,70038202360,41.844723075865055,b19c635d-29f8-4f7...,2022-08-09,Varius Orci Insti...,tent and awning s...,6.3,a,7276,1226530,Victoria Gonzalez,68657 Johnson Gle...,TAS,Female,7276,108.99254564781535,146.9635580479313,-41.28259951036976,"26,537",null,null
19,76275088055,21.068973724951874,3cb22c6f-2a3a-47c...,2022-08-10,Vitae Risus Indus...,"opticians, optica...",4.74,b,7276,1226530,Victoria Gonzalez,68657 Johnson Gle...,TAS,Female,7276,108.99254564781535,146.9635580479313,-41.28259951036976,"27,284",null,null


22/10/08 14:32:02 WARN TransportChannelHandler: Exception in connection from /100.70.11.210:60585
java.io.IOException: Operation timed out
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:258)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:722)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOpt